# Comparing the Original EPC Dataset with the Cleansed Version

In [1]:
import pandas as pd
from ipywidgets import interact

from epc_data_analysis import PROJECT_DIR
from epc_data_analysis.getters import epc_data, util_data
from epc_data_analysis.analysis.notebooks.notebook_utils import my_widgets
from epc_data_analysis.pipeline import feature_engineering, easy_plotting, data_cleaning

## Getting the data

The original data is made to match the cleansed set: only England and Wales data and de-duplicated. 

The preprocessed original data can be loaded directly or processed here. 

In [2]:
use_preprocessed_original_data = True
save_preprocessed_data = False

DIR_PATH = str(PROJECT_DIR)+'/inputs/Comparing_datasets/'

### Loading the cleansed data

In [3]:
cleansed_epc_df = pd.read_csv(DIR_PATH+'EPC_Records__cleansed_and_deduplicated.csv', low_memory=True)

cleansed_epc_df["YEAR"] = cleansed_epc_df["LODGEMENT_DATE"].apply(
            feature_engineering.get_year)

/Users/juliasuter/opt/anaconda3/envs/EPC_data_analysis/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (8,9,11,37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Option 1) Loading the preprocessed orignal data

In [4]:
if use_preprocessed_original_data:
    original_epc_df = pd.read_csv(DIR_PATH+"Original_EPC_Wales_England.csv")
    original_epc_df_non_depl = pd.read_csv(DIR_PATH+"Original_EPC_Wales_England_non_depupl.csv")

### Option 2) Preprocessing the original data

In [5]:
if not use_preprocessed_original_data:

    features_of_interest = ['ADDRESS1', 'CURRENT_ENERGY_RATING', 'POTENTIAL_ENERGY_RATING', 'TENURE',
                            'CURRENT_ENERGY_EFFICIENCY', 
                            'LMK_KEY', 'POSTCODE', 'MAINHEAT_DESCRIPTION', 'CO2_EMISSIONS_CURRENT', 
                            'CO2_EMISS_CURR_PER_FLOOR_AREA', 'LIGHTING_COST_CURRENT', 'HEATING_COST_CURRENT',
                            'HOT_WATER_COST_CURRENT', 'BUILDING_REFERENCE_NUMBER', 'LODGEMENT_DATE', 
                            'INSPECTION_DATE', 'BUILT_FORM', 'PROPERTY_TYPE', 'CONSTRUCTION_AGE_BAND',
                            'TRANSACTION_TYPE', 'MAIN_FUEL', 'TOTAL_FLOOR_AREA', 'ENERGY_TARIFF']


    epc_Wales = epc_data.load_epc_data(
        subset='Wales', usecols=features_of_interest, low_memory=False
    )

    epc_England = epc_data.load_epc_data(
        subset='England', usecols=features_of_interest, low_memory=False
    )

    print('Wales: {} samples'.format(epc_Wales.shape[0]))
    print('England: {} samples'.format(epc_England.shape[0]))
    original_epc_df = pd.concat([epc_Wales, epc_England], axis=0)

    print('Combined: {} samples'.format(original_epc_df.shape[0]))
    
    print('------------------')
    
    # Preprocessing
    original_epc_df_non_depl = data_cleaning.clean_epc_data(original_epc_df)
    print(original_epc_df_non_depl.shape)

    original_epc_df_non_depl = feature_engineering.get_additional_features(original_epc_df)  
    print(original_epc_df_non_depl.shape)
    
    original_epc_df = feature_engineering.filter_by_year(original_epc_df_non_depl, 'BUILDING_ID', None, selection="latest entry")
    print(original_epc_df.shape)

In [6]:
if save_preprocessed_data:
    original_epc_df.to_csv(DIR_PATH+'Original_EPC_Wales_England.csv', index=False)
    original_epc_df_non_depl.to_csv(DIR_PATH+'Original_EPC_Wales_England_non_depupl.csv', index=False)

## Analysis

### Sample comparison

In [7]:
print('Original: {} samples'.format(original_epc_df.shape[0]))
print('Cleansed: {} samples'.format(cleansed_epc_df.shape[0]))
print('-------------------')
print('Original: {} features\nDepends on selection of features.'.format(original_epc_df.shape[1]))
print()
print('Cleansed: {} features'.format(cleansed_epc_df.shape[1]))

Original: 15631334 samples
Cleansed: 14343966 samples
-------------------
Original: 35 features
Depends on selection of features.

Cleansed: 44 features


### Comparing distributions (normalised)

In [8]:
column_widget_1 = my_widgets.get_custom_widget(
    original_epc_df.columns,
    description="Original EPC feature",
    default_value="CURRENT_ENERGY_RATING",
    widget_type="dropdown",
)

column_widget_2 = my_widgets.get_custom_widget(
    cleansed_epc_df.columns,
    description="Cleansed EPC feature",
    default_value="FINAL_EPC_BAND",
    widget_type="dropdown",
)


print("Original EPC Data")
@interact(category=column_widget_1)
def plot_distribution_normalised(category):
    easy_plotting.plot_subcategory_distribution(
        original_epc_df, category, normalize=True, y_label="Properties", x_tick_rotation=45,
        plot_title="Original EPC: Distribution for {} (%)".format(category)
    )
    
print("Cleansed EPC Data")
@interact(category=column_widget_2)
def plot_distribution_normalised(category):
    easy_plotting.plot_subcategory_distribution(
        cleansed_epc_df, category, normalize=True, y_label="Properties", x_tick_rotation=45,
        plot_title="Cleansed EPC: Distribution for {} (%)".format(category)
    )

Original EPC Data


interactive(children=(Dropdown(description='Original EPC feature', index=4, options=('LMK_KEY', 'ADDRESS1', 'P…

Cleansed EPC Data


interactive(children=(Dropdown(description='Cleansed EPC feature', index=28, options=('Unnamed: 0', 'ROW_NUM',…

### Comparing distributions (not normalised)

In [9]:
column_widget_1 = my_widgets.get_custom_widget(
    original_epc_df.columns,
    description="EPC feature",
    default_value="CURRENT_ENERGY_RATING",
    widget_type="dropdown",
)

column_widget_2 = my_widgets.get_custom_widget(
    cleansed_epc_df.columns,
    description="EPC feature",
    default_value="FINAL_EPC_BAND",
    widget_type="dropdown",
)


print("Original EPC Data")
@interact(category=column_widget_1)
def plot_distribution(category):
    easy_plotting.plot_subcategory_distribution(
        original_epc_df, category, normalize=False, y_label="Properties", x_tick_rotation=45,y_ticklabel_type='k',
        plot_title="Original EPC: Distribution for {}".format(category)
    )
    
print("Cleansed EPC Data")
@interact(category=column_widget_2)
def plot_distribution(category):
    easy_plotting.plot_subcategory_distribution(
        cleansed_epc_df, category, normalize=False, y_label="Properties", x_tick_rotation=45, y_ticklabel_type='k',
        plot_title="Cleansed EPC: Distribution for {}".format(category)
    )

Original EPC Data


interactive(children=(Dropdown(description='EPC feature', index=4, options=('LMK_KEY', 'ADDRESS1', 'POSTCODE',…

Cleansed EPC Data


interactive(children=(Dropdown(description='EPC feature', index=28, options=('Unnamed: 0', 'ROW_NUM', 'LMK_KEY…

## What data is kept? The latest entry?

Get properties with several buildings and check lodgement date (and for double checking also EPC score). 

In [10]:
original_epc_df[original_epc_df['N_ENTRIES_BUILD_ID'] > 1]['BUILDING_REFERENCE_NUMBER']

1           7194793278
2           3301443078
4           7990446478
10          8347289178
14          3573995478
               ...    
15631319    2001030278
15631329    2424151568
15631330    9661627568
15631332    2671832468
15631333    5936297078
Name: BUILDING_REFERENCE_NUMBER, Length: 3477938, dtype: int64

In [11]:
building_reference_example = [4649558968, 7194793278, 8347289178, 9661627568, 5936297078][2]

In [12]:
original_epc_df_non_depl[original_epc_df_non_depl['BUILDING_REFERENCE_NUMBER'] == building_reference_example][['CURRENT_ENERGY_EFFICIENCY', 'LODGEMENT_DATE', 'LMK_KEY']]

,CURRENT_ENERGY_EFFICIENCY,LODGEMENT_DATE,LMK_KEY
18080494,13,2018-05-24,1091757889342018052415024617982328
18080531,1,2014-02-17,1091757882352014021718203298040813


In [13]:
original_epc_df[original_epc_df['BUILDING_REFERENCE_NUMBER'] == building_reference_example][['CURRENT_ENERGY_EFFICIENCY', 'LODGEMENT_DATE', 'LMK_KEY']]

,CURRENT_ENERGY_EFFICIENCY,LODGEMENT_DATE,LMK_KEY
10,13,2018-05-24,1091757889342018052415024617982328


In [14]:
cleansed_epc_df[cleansed_epc_df['BUILDING_REFERENCE_NUMBER'] == building_reference_example][['FINAL_EPC_SCORE','LODGEMENT_DATE', 'LMK_KEY' ]]

,FINAL_EPC_SCORE,LODGEMENT_DATE,LMK_KEY
2830757,13.0,2018-05-24,1091757889342017982642440260466042


### Final Fuel Bill Composision

FINAL_FUEL_BILL seems to be combination of HEATING_COST_CURRENT, HOT_WATER_COST_CURRENT and LIGHTING_COST_CURRENT.

In [15]:
building_reference_number = 9637549468

original_sample = original_epc_df[original_epc_df['BUILDING_REFERENCE_NUMBER'] == building_reference_number]
cleansed_sample = cleansed_epc_df[cleansed_epc_df['BUILDING_REFERENCE_NUMBER'] == building_reference_number]


print(original_sample['HEATING_COST_CURRENT'])
print(original_sample['HOT_WATER_COST_CURRENT'])
print(original_sample['LIGHTING_COST_CURRENT'])

print(cleansed_sample['FINAL_FUEL_BILL'])

print('----------------------')
print('Summed up costs:', original_sample['HEATING_COST_CURRENT'] + original_sample['HOT_WATER_COST_CURRENT'] + original_sample['LIGHTING_COST_CURRENT'])

457799    2088.0
Name: HEATING_COST_CURRENT, dtype: float64
457799    598.0
Name: HOT_WATER_COST_CURRENT, dtype: float64
457799    87.0
Name: LIGHTING_COST_CURRENT, dtype: float64
32372    2773.0
Name: FINAL_FUEL_BILL, dtype: float64
----------------------
Summed up costs: 457799    2773.0
dtype: float64


### What happened to the unknown sector and other NODATA! entries?

What happened to samples with unknown tenure type or other missing data?

Some data could be filled in, other samples were discarded.

```
Tenure Type:  0.2% filled in
Construction Age: 0.6% filled in
Built Form: 70% filled in
```

In [16]:
unknown_tenure_keys = list(original_epc_df[original_epc_df['TENURE'] == 'unknown']['BUILDING_REFERENCE_NUMBER'])
unknown_age_keys = list(original_epc_df[original_epc_df['CONSTRUCTION_AGE_BAND'] == 'unknown']['BUILDING_REFERENCE_NUMBER'])
unknown_built_keys = list(original_epc_df[original_epc_df['BUILT_FORM'] == 'NO DATA!']['BUILDING_REFERENCE_NUMBER'])

print('Number of unknown data samples for:')
print('Tenure: {}'.format(len(unknown_tenure_keys)))
print('Construction Age: {}'.format(len(unknown_age_keys)))
print('Built Form: {}'.format(len(unknown_built_keys)))

cleansed_epc_df[cleansed_epc_df['BUILDING_REFERENCE_NUMBER'].isin(unknown_tenure_keys)]["FINAL_PROP_TENURE"].value_counts()

Number of unknown data samples for:
Tenure: 1454267
Construction Age: 2113967
Built Form: 425316


Owner Occupied      1713
Social              1507
Privately Rented     991
Name: FINAL_PROP_TENURE, dtype: int64

In [17]:
cleansed_epc_df[cleansed_epc_df['BUILDING_REFERENCE_NUMBER'].isin(unknown_age_keys)]["FINAL_PROPERTY_AGE"].value_counts()

1950_1966    292
Pre_1900     243
1900_1929    230
Post_1996    228
1930_1949    226
1967_1982    188
1983_1995     37
Name: FINAL_PROPERTY_AGE, dtype: int64

In [18]:
cleansed_epc_df[cleansed_epc_df['BUILDING_REFERENCE_NUMBER'].isin(unknown_built_keys)]["FINAL_PROPERTY_TYPE"].value_counts()

Block of flats         299657
Detached house             29
Semi-detached house        25
End-terraced house         13
Mid-terraced house          8
Name: FINAL_PROPERTY_TYPE, dtype: int64

### LMK_KEYS are differnet for two datasets

There seems to be no overlap, even on the newer data. 

In [19]:
cleansed_epc_df[cleansed_epc_df['BUILDING_REFERENCE_NUMBER'] == 4504250668][['LMK_KEY', 'ADDRESS1', 'POSTCODE']]

,LMK_KEY,ADDRESS1,POSTCODE
41236,264982459542014075844282626080068,"28, Joicey Court",TS26 8BZ


In [20]:
original_epc_df[original_epc_df['BUILDING_REFERENCE_NUMBER'] == 4504250668][['LMK_KEY', 'ADDRESS1', 'POSTCODE']]

,LMK_KEY,ADDRESS1,POSTCODE
5314824,264982459542014092909494760049908,"28, Joicey Court",TS26 8BZ


In [21]:
LMK_KEY_example = '460450212302020031014402078409708'
#LMK_KEY_example = '460450257032010032507140438268803'

In [22]:
original_epc_df[original_epc_df['LMK_KEY'] == str(LMK_KEY_example)]

,LMK_KEY,ADDRESS1,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,PROPERTY_TYPE,BUILT_FORM,INSPECTION_DATE,...,DATE_INT,UNIQUE_ADDRESS,BUILDING_ID,N_ENTRIES,N_ENTRIES_BUILD_ID,HEATING_SYSTEM,HEATING_SOURCE,CURR_ENERGY_RATING_NUM,ENERGY_RATING_CAT,DIFF_POT_ENERGY_RATING
1395029,460450212302020031014402078409708,"2, Amberwood Walk",TS27 3QG,5368814768,C,B,70,House,Mid-Terrace,2020-03-10,...,20200310,"2, Amberwood WalkTS27 3QG",2590081647001208,2,2,boiler and radiator,gas,5,C-D,1


In [23]:
original_epc_df_non_depl[original_epc_df_non_depl['BUILDING_REFERENCE_NUMBER'] == 5368814768]['LMK_KEY']

10806189    460450212302020031014402078409708
10828762    460450257032010032507140438268803
Name: LMK_KEY, dtype: object

In [24]:
cleansed_epc_df[cleansed_epc_df['LMK_KEY'] == LMK_KEY_example]

,Unnamed: 0,ROW_NUM,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,POSTCODE,BUILDING_REFERENCE_NUMBER,LOCAL_AUTHORITY,LOCAL_AUTHORITY_LABEL,...,FINAL_FLOOR_INS,FINAL_HEAT_CONTROL,FINAL_LOW_ENERGY_LIGHTING,FINAL_FIREPLACES,FINAL_WIND_FLAG,FINAL_PV_FLAG,FINAL_SOLAR_THERMAL_FLAG,FINAL_MAIN_FUEL_NEW,FINAL_HEATING_SYSTEM,YEAR


In [25]:
cleansed_epc_df[cleansed_epc_df['LMK_KEY'] == int(LMK_KEY_example)]

,Unnamed: 0,ROW_NUM,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,POSTCODE,BUILDING_REFERENCE_NUMBER,LOCAL_AUTHORITY,LOCAL_AUTHORITY_LABEL,...,FINAL_FLOOR_INS,FINAL_HEAT_CONTROL,FINAL_LOW_ENERGY_LIGHTING,FINAL_FIREPLACES,FINAL_WIND_FLAG,FINAL_PV_FLAG,FINAL_SOLAR_THERMAL_FLAG,FINAL_MAIN_FUEL_NEW,FINAL_HEATING_SYSTEM,YEAR


### LMK_KEYS are differnet for two datasets - What about the 2020 data?

There seems to be no overlap, even on the newer data. 

In [26]:
cleansed_2020 = list(cleansed_epc_df[cleansed_epc_df['YEAR'] == 2020]['LMK_KEY'])

In [27]:
original_2020 = list(original_epc_df[original_epc_df['YEAR'] == 2020]['LMK_KEY'])

In [28]:
original_2020_non_dedupl = list(original_epc_df_non_depl[original_epc_df_non_depl['YEAR'] == 2020]['LMK_KEY'])

In [29]:
print(len(cleansed_2020))
print(len(original_2020))
print(len(original_2020_non_dedupl))

360004
975117
994127


In [31]:
#for key in cleansed_2020:
#    if key in original_2020_non_dedupl:
#        print(key)

In [32]:
original_epc_df[original_epc_df['LMK_KEY'] == '918812462502020072622401808702468']

,LMK_KEY,ADDRESS1,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,PROPERTY_TYPE,BUILT_FORM,INSPECTION_DATE,...,DATE_INT,UNIQUE_ADDRESS,BUILDING_ID,N_ENTRIES,N_ENTRIES_BUILD_ID,HEATING_SYSTEM,HEATING_SOURCE,CURR_ENERGY_RATING_NUM,ENERGY_RATING_CAT,DIFF_POT_ENERGY_RATING
16,918812462502020072622401808702468,"18, Castle Road",RH12 3NB,588167078,C,B,71,House,Semi-Detached,2020-07-26,...,20200726,"18, Castle RoadRH12 3NB",1970048004028869,2,2,boiler and radiator,gas,5,C-D,1


In [33]:
cleansed_epc_df[cleansed_epc_df['BUILDING_REFERENCE_NUMBER'] == 588167078]

,Unnamed: 0,ROW_NUM,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,POSTCODE,BUILDING_REFERENCE_NUMBER,LOCAL_AUTHORITY,LOCAL_AUTHORITY_LABEL,...,FINAL_FLOOR_INS,FINAL_HEAT_CONTROL,FINAL_LOW_ENERGY_LIGHTING,FINAL_FIREPLACES,FINAL_WIND_FLAG,FINAL_PV_FLAG,FINAL_SOLAR_THERMAL_FLAG,FINAL_MAIN_FUEL_NEW,FINAL_HEATING_SYSTEM,YEAR
8199070,8199071,10857782,918812404732012993924666022082802,"18, Castle Road",Broadbridge Heath,NaN,RH12 3NB,588167078,E07000227,Horsham,...,UnInsulated,Programmer and thermostat,0-25%,0,No,No,NaN,Mains Gas,Boiler,2013


In [34]:
original_epc_df[original_epc_df['LMK_KEY'] == '1809118592242020070922512673000818']

,LMK_KEY,ADDRESS1,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,PROPERTY_TYPE,BUILT_FORM,INSPECTION_DATE,...,DATE_INT,UNIQUE_ADDRESS,BUILDING_ID,N_ENTRIES,N_ENTRIES_BUILD_ID,HEATING_SYSTEM,HEATING_SOURCE,CURR_ENERGY_RATING_NUM,ENERGY_RATING_CAT,DIFF_POT_ENERGY_RATING
2456,1809118592242020070922512673000818,"82, Albert Park Road",WR14 1RR,2215090778,E,C,46,House,Semi-Detached,2020-07-09,...,20200709,"82, Albert Park RoadWR14 1RR",2609284596355912,1,1,boiler and radiator,gas,3,E-G,2


In [35]:
cleansed_epc_df[cleansed_epc_df['LMK_KEY'] == '1809118592242020070922512673000818']

,Unnamed: 0,ROW_NUM,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,POSTCODE,BUILDING_REFERENCE_NUMBER,LOCAL_AUTHORITY,LOCAL_AUTHORITY_LABEL,...,FINAL_FLOOR_INS,FINAL_HEAT_CONTROL,FINAL_LOW_ENERGY_LIGHTING,FINAL_FIREPLACES,FINAL_WIND_FLAG,FINAL_PV_FLAG,FINAL_SOLAR_THERMAL_FLAG,FINAL_MAIN_FUEL_NEW,FINAL_HEATING_SYSTEM,YEAR
